In [1]:
tag_of_word = {}
words_with_tag = {}
with open('./dane/supertags.txt', 'r', encoding='utf-8') as file:
    for line in file:
        word, tag = line.lower().rstrip('\n').split()
        tag_of_word[word] = tag
        if tag in words_with_tag:
            words_with_tag[tag].append(word)
        else:
            words_with_tag[tag] = [word]

In [2]:
bigram = {}
with open('./dane/poleval_2grams.txt', 'r', encoding='utf-8') as file:
    for line in file:
        n, word1, word2 = line.lower().split()
        n = int(n)
        if n < 10: 
            continue
        if word1 in bigram:
            bigram[word1].append((word2, n))
        else:
            bigram[word1] = [(word2, n)]

In [3]:
from collections import defaultdict

unigram = {}
unigram = defaultdict(lambda: 0, unigram)
for word1 in bigram:
    for word2, n in bigram[word1]:
        unigram[word1] += n
        unigram[word2] += n

In [4]:
def get_tag(word):
    if word in tag_of_word:
        return tag_of_word[word]
    return tag_of_word[('^' + word)[-3:]]

In [5]:
from random import choices


def get_unigram_probability(word):
    if word in unigram:
        return unigram[word]
    return 0.0001


def choose_from_unigram(words):
    weights = [get_unigram_probability(x) for x in words]
    return choices(words, weights=weights, k=1)[0]


def get_tag(word):
    if word in tag_of_word:
        return tag_of_word[word]
    return tag_of_word[('^' + word)[-3:]]


def random_similar_sentence(original):
    words = original.lower().split()
    tags = list(map(get_tag, words))
    alternative_words = list(map(lambda tag: words_with_tag[tag], tags))
    chosen_words = list(map(choose_from_unigram, alternative_words))
    return ' '.join(chosen_words)

In [6]:
random_similar_sentence("Lubię obserwować ładne ptaki w piękne słoneczne dni w lesie")

'lubię pełnić dodatkowe psy w liczne kolejne dni w czasie'

In [7]:
sentences = [
    "Litwo Ojczyzno moja Ty jesteś jak zdrowie",
    "Jeden z pojmanych najemników probował odebrać to ostrze",
    "Jak może być inaczej dopóki gildia znajduje się poza imperialną kontrolą",
    "Jak sobie życzysz",
    "Książę popatrzył spode łba i zajął się sterami",
]

In [8]:
for sentence in sentences:
    print(random_similar_sentence(sentence))

eureko izbo sama ty jesteś jak bezpieczeństwo
jeden z znanych krytyków obóz zrealizować to rozwiązanie
jak może być wyłącznie żeby rola brakuje się poza znaczną recesją
jak sobie możesz
książę wystartował spode grudnia i utracił się dokumentami


In [14]:
def get_decent_combination(combinations):
    acceptable_combinations = []
    acceptable_combinations_weights = []
    for word1, word2 in combinations:
        if word1 in bigram:
            for successor, n in bigram[word1]:
                if word2 == successor:
                    acceptable_combinations.append((word1, word2))
                    acceptable_combinations_weights.append(n)
    if len(acceptable_combinations) == 0:
        return None
    return choices(acceptable_combinations, weights=acceptable_combinations_weights, k=1)[0]


def random_similar_sentence_v2(original):
    words = original.lower().split()
    first_word_alternatives = words_with_tag[get_tag(words[0])]
    second_word_alternatives = words_with_tag[get_tag(words[1])]
    
    combinations = [(alt1, alt2) for alt1 in first_word_alternatives for alt2 in second_word_alternatives]
    word1, word2 = get_decent_combination(combinations)
    prev = word1
    sentence = [word1]
    for word in words[1:]:
        word_alternatives = words_with_tag[get_tag(word)]
        combinations = [(prev, alt) for alt in word_alternatives]
        combination = get_decent_combination(combinations)

        if combination is not None:
            _, prev = combination
        else:
            sentence.append('|')
            prev = choose_from_unigram(word_alternatives)
            
        sentence.append(prev)
    
    return ' '.join(sentence)


In [10]:
random_similar_sentence_v2("Lubię obserwować ładne ptaki w piękne słoneczne dni w lesie")

'lubię robić własne psy na małe ciche dni przed | języku'

In [12]:
sentences = [
    "Jeden z pojmanych najemników probował odebrać to ostrze",
    "Jak może być inaczej dopóki gildia znajduje się poza imperialną kontrolą",
    "Jak sobie życzysz",
    "Książę popatrzył spode łba i zajął się sterami",
]

In [15]:
for sentence in sentences:
    print(random_similar_sentence_v2(sentence))

jeden z wprowadzonych | rodziców obowiązek poinformować | to wiązanie
jak może być zupełnie natomiast idea wydaje się poza wielką miłośniczką
jak sobie możesz
książę zmarł | spode łba i wprowadził się przekładami


In [18]:
print('natomiast' in list(map(lambda x: x[0], bigram['zupełnie'])))

True


In [21]:
print(random_similar_sentence_v2("Bawiąc się widelcem patrzył z namysłem na niebo"))

powołując się przedmiotem | uczęszczał | z tytułem na pozostawienie


In [22]:
print(random_similar_sentence_v2("W paru miejscach za stołem rozległy się chichoty"))

w wielu posiedzeniach | za rogiem | rozległy się sklepy


In [24]:
print(random_similar_sentence_v2("Drzwiami służbowymi wpadł umundurowany żołnierz i pospieszył do księcia"))

finansami publicznymi | osiągnął | miniony | pan i wstąpił do paryża


In [25]:
print(random_similar_sentence_v2("Nie ma tego złego co by na dobre nie wyszło"))

nie ma tego rozległego | co by po kolejne nie doszło
